In [2]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import numpy as np # You'll likely need this one too

In [23]:
df = pd.read_excel("WebTraffic.xlsx")

In [24]:
# 1. Date Conversion: Convert VisitDate to datetime
df['VisitDate'] = pd.to_datetime(df['VisitDate'])

# 2. Bounce Rate Scaling: Assuming BounceRate is a percentage, convert it to a ratio (0 to 1).
# We'll check if the values are generally > 1, and only scale if they are.
# If they are already ratios (0 to 1), this operation will do no harm.
if (df['BounceRate'] > 1).any():
    print("Scaling BounceRate (Dividing by 100)...")
    df['BounceRate'] = df['BounceRate'] / 100.0

# Ensure BounceRate is within a valid ratio range [0, 1] after scaling
df['BounceRate'] = df['BounceRate'].clip(lower=0, upper=1)

# 3. Feature Engineering: Create Engagement Score
# Formula: (SessionDurationSeconds * PageViews) / BounceRate (or 1 if BounceRate is 0)
# We use numpy.where to prevent division by zero.
import numpy as np
df['EngagementScore'] = (df['SessionDurationSeconds'] * df['PageViews']) / np.where(df['BounceRate'] == 0, 1, df['BounceRate'])

# Check the results of the transformations
print("\n--- Transformed Data Head ---")
print(df.head())
print("\n--- Final Data Info ---")
df.info()

Scaling BounceRate (Dividing by 100)...

--- Transformed Data Head ---
  SessionID UserID  VisitDate   Device  PageViews  BounceRate  \
0        S0     U0 2024-11-12   Tablet          8      0.7400   
1        S1     U1 2024-07-15   Mobile          7      0.3098   
2        S2     U2 2024-09-25   Tablet         11      0.8163   
3        S3     U3 2025-02-11   Mobile         10      0.0544   
4        S4     U4 2025-04-27  Desktop          4      0.2011   

   SessionDurationSeconds  EngagementScore  
0                     909      9827.027027  
1                     620     14009.038089  
2                     920     12397.402916  
3                     553    101654.411765  
4                    1750     34808.552959  

--- Final Data Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   SessionID  

In [25]:
# --- A. Analyze average SessionDurationSeconds by Device (Your Task) ---
# Replace df with data in your notebook!
device_duration_avg = df.groupby('Device')['SessionDurationSeconds'].mean().sort_values(ascending=False)
print("\n--- A. Average Session Duration by Device (seconds) ---")
print(device_duration_avg)

# --- B. Correlate session duration with PageViews and BounceRate (Your Task) ---
# Replace df with data in your notebook!
correlation_matrix = df[['SessionDurationSeconds', 'PageViews', 'BounceRate']].corr()
print("\n--- B. Correlation Matrix ---")
print(correlation_matrix)


--- A. Average Session Duration by Device (seconds) ---
Device
Desktop    923.553127
Tablet     914.294223
Mobile     901.794803
Name: SessionDurationSeconds, dtype: float64

--- B. Correlation Matrix ---
                        SessionDurationSeconds  PageViews  BounceRate
SessionDurationSeconds                1.000000   0.014201    0.000591
PageViews                             0.014201   1.000000   -0.001320
BounceRate                            0.000591  -0.001320    1.000000


In [26]:
# Save the final cleaned and augmented data for SQL import
# Replace df with data in your notebook!
df.to_csv('final_web_traffic_for_sql.csv', index=False)
print("Exported 'final_web_traffic_for_sql.csv' for SQL import.")

Exported 'final_web_traffic_for_sql.csv' for SQL import.
